## Technical requirements

```bash
conda create -n gee python
conda activate gee
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
```


In [ ]:
#%pip install pygis

In [1]:
import ee
import geemap
import leafmap
#https://medium.com/@geonextgis/getting-started-with-geemap-a-guide-to-working-with-feature-and-image-collections-dfa438d59621

In [2]:
geemap.ee_initialize()

## Load data

In [178]:
WorldPop = ee.ImageCollection("WorldPop/GP/100m/pop")
pop_sahel = ee.Image("projects/ee-aboubacarhema94/assets/ACLED/Sahel_pop_2020_UNadj_constrained")
sahel_adm2 = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/ACLED/G5_Sahel_adm2")
events_diffusion = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/ACLED/G5_Sahel_Diffusion")
geometry = ee.Geometry.Polygon(
        [[[-19.7019400534855, 29.93665627910383],
          [-19.7019400534855, 5.813630636937255],
          [26.967981821514496, 5.813630636937255],
          [26.967981821514496, 29.93665627910383]]])

pop_sahel = ee.Image(pop_sahel)

In [179]:
viz_pop = {
  'min': 0.0,
  'max': 250.0,
  'palette': ['24126c', '1fff4f', 'd4ff50'],
}

In [180]:
#Extract the projection before doing any computation
projection = pop_sahel.projection()
#print(ee.Number(projection.nominalScale()))

In [181]:
Map = geemap.Map()
# Creat an interactive map
#Map = Map.addLayer(WorldPop, viz_pop, 'Population Sahel', 0);
Map.addLayer(pop_sahel, viz_pop, 'Population Sahel', True, 1)
Map.addLayer(sahel_adm2, {'color': 'purple'}, "sahel adm2")
Map.centerObject(sahel_adm2)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [182]:
pop_sahel_stats = 'pop_sahel_stats.csv'
geemap.zonal_stats(
    pop_sahel,
    sahel_adm2,
    pop_sahel_stats,
    statistics_type='SUM',
    scale=100, 
    return_fc=False
)


Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\AHema\OneDrive - CGIAR\Desktop\2023\NORAD\Political Violence in the G5 Sahel Countries\Conflict diffusion indicator\scripts\pop_sahel_stats.csv


## Start conflict diffusion indicator computation

In [183]:

# year of event
year = 2018

#
pop_seuil = 50

#
nb_events = 3

1. Create a spatial grid of 10km-by-10km

In [184]:
# ****** Aggregation to 10km ****** #
country_pop = pop_sahel

# Get the projection at required scale
projectionAt1k = projection.atScale(1000)
projectionAt10k = projection.atScale(10000)

# Step1: 100m to 1000m
country_popAt1k = country_pop \
  .reduceResolution(**{
    'reducer': ee.Reducer.sum().unweighted(),
    'maxPixels': 1024
  }) \
  .reproject(**{
    'crs': projectionAt1k
  })
# Step2: 1000m to 10000m
country_popAt10k = country_popAt1k \
  .reduceResolution(**{
    'reducer': ee.Reducer.sum().unweighted(),
    'maxPixels': 1024
  }) \
  .reproject(**{
    'crs': projectionAt10k
  })

In [185]:
Map.addLayer(country_popAt1k.clip(sahel_adm2), viz_pop, 'Population Sahel 1km', 1)
Map.addLayer(country_popAt10k.clip(sahel_adm2), viz_pop, 'Population Sahel 10km', 1)
Map

Map(bottom=3995.0, center=[17.369569793340958, 3.4630404598133353], controls=(WidgetControl(options=['position…

2. Remove grid cells with less than 100 habitants using population data from WorldPop (pop_country)

In [186]:
country_popAt10k_100 = country_popAt10k.gte(pop_seuil)
country_popAt10k_100 = country_popAt10k_100.updateMask(country_popAt10k_100.neq(0))
bin = {'min': 0, 'max': 1, 'palette': ['red', 'green']}
Map.addLayer(country_popAt10k_100, bin, 'Population 10km gt ' , 1)#'Population 10km gt ' + pop_seuil + ' habitants'
Map


Map(bottom=3995.0, center=[17.369569793340958, 3.4630404598133353], controls=(WidgetControl(options=['position…

In [187]:
country_popAt10k_100 = country_popAt10k_100.unmask(0).rename('grid_cells_gte')#'grid_cells_gte' + pop_seuil + 'habitants'

3. Identify grid cells with at least 10 conflict events per year using events_diffusion layer

3.1 Number of Conflict event

In [136]:
'''
import geemap.chart as chart

features = events_diffusion.select('event_type')
df = geemap.ee_to_df(features, sort_columns=True)
df
'''

"\nimport geemap.chart as chart\n\nfeatures = events_diffusion.select('event_type')\ndf = geemap.ee_to_df(features, sort_columns=True)\ndf\n"

In [188]:
events_diffusion_country_year = events_diffusion \
                                              .filter(ee.Filter.eq('year', year))

#calculating the number of non-null values


3.2 Vizualise Conflict event layer

In [189]:
# Use style() to visualize the points
eventsStyled = {
    'color': 'red',
    'pointSize': 1,
    'pointShape': 'triangle',
    'width': 3,
}

Map.addLayer(events_diffusion_country_year.style(**eventsStyled), {}, "conflict events")
Map

Map(bottom=3995.0, center=[17.369569793340958, 3.4630404598133353], controls=(WidgetControl(options=['position…

3.3 Create image from conflict events layer



In [190]:
#/ add dummy property to use for reduceToImage

def func_dlg(feature):
  return feature.set('dummy',1)

if_events = ee.FeatureCollection(events_diffusion_country_year).map(func_dlg)


In [191]:
eventsImg = if_events.reduceToImage(['dummy'], ee.Reducer.sum().unweighted()) \
  .unmask(0) \
  .reproject('epsg:4326', None, 10000) \
  .clip(geometry)


In [192]:
events_sahel_stats = 'events_sahel_stats.csv'
geemap.zonal_stats(
    eventsImg,
    sahel_adm2,
    events_sahel_stats,
    statistics_type='SUM',
    scale=10000, 
    return_fc=False
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\AHema\OneDrive - CGIAR\Desktop\2023\NORAD\Political Violence in the G5 Sahel Countries\Conflict diffusion indicator\scripts\events_sahel_stats.csv


In [193]:
viz_events = {
  'min': 0.0,
  'max': 34.0,
  'palette': [
    "00ff00","1A492C","071EC4","B5CA36","729EAC","8EA5DE",
    "818991","62A3C3","CCF4FE","74F0B9","yellow","C72144",
    "56613B","C14683","C31C25","5F6253","11BF85","A61B26",
    "99FBC5","188AAA","C2D7F1","B7D9D8","856F96","109C6B",
    "2DE3F4","9A777D","151796","C033D8","510037","640C21",
    "31A191","223AB0","B692AC","2DE3F4",
  ]
}
eventsImg_viz = eventsImg.updateMask(eventsImg.neq(0))
Map.addLayer(eventsImg_viz.clip(sahel_adm2), viz_events , 'Number of Conflict event',1)
Map

Map(bottom=3995.0, center=[17.369569793340958, 3.4630404598133353], controls=(WidgetControl(options=['position…

In [194]:
multiplication = country_popAt10k_100.multiply(eventsImg)
multiplication_10 = multiplication.gte(nb_events)
multiplication_10 = multiplication_10.updateMask(multiplication_10.neq(0))
Map.addLayer(multiplication_10.clip(sahel_adm2), bin, 'grid cells needed', 1)
Map

Map(bottom=3995.0, center=[17.369569793340958, 3.4630404598133353], controls=(WidgetControl(options=['position…

In [195]:
multiplication_10 = multiplication_10.unmask(0).rename('grid_cells_needed')


In [200]:
stacked_image = multiplication_10.addBands(country_popAt10k_100)
#Create a function to calculate the feature class with ADM2 Name
def calculateFeatureSum(feature):
    events = stacked_image.reduceRegion(**{
    'reducer': ee.Reducer.sum().unweighted(),
    'geometry': feature.geometry(),#.buffer(10),
    'scale': projectionAt10k.nominalScale(),
    'maxPixels': 1e20
    })
    adm_level = feature.get('admin2Pcod')
    return ee.Feature(
      feature.geometry(),
      events.set('admin2Pcod', adm_level))

In [201]:
#Map Function to Create
Feature_byADM2 = sahel_adm2.map(calculateFeatureSum)
#geemap.ee_to_csv(Feature_byADM2, filename='conflict_diffusion_indicator.csv')

4. Compute the proportion of high violence grid cells to total cells per adm2/year

In [202]:
def indicator(feature):
  #
  val = ee.Number(feature.get('grid_cells_needed')).divide(ee.Number(feature.get('grid_cells_gte')))
  return feature.set('conflict_diffusion_indicator', val)

Feature_byADM2 = Feature_byADM2.map(indicator)

## Export to CSV

In [203]:
geemap.ee_to_csv(Feature_byADM2, filename='conflict_diffusion_indicator.csv')

In [154]:
'''
snippet = """
Export.table.toDrive({
    collection: Feature_byADM2,
    fileNamePrefix: 'events_diffusion_Count' + country + year + 'grid_cells_gte' + pop_seuil + 'habitants_gte' + nb_events,
    description: 'events_diffusion_Count' + country + year + 'grid_cells_gte' + pop_seuil + 'habitants_gte' + nb_events ,
    //folder: "", //set based on user preference
    fileFormat: 'CSV',
    selectors: ['admin2Pcod', 'grid_cells_gte' + pop_seuil + 'habitants_gte' + nb_events + 'events','grid_cells_gte' + pop_seuil + 'habitants','conflict_diffusion_indicator']
    });
 
"""

geemap.js_snippet_to_py(snippet, add_new_cell=True, import_ee=False)
'''